In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm 
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
%matplotlib inline

# Evaluating the House prices regression model

In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
prices_df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [4]:
#Create series of continuous variable names, check that all continuous variables are numerical

cont_vars = pd.Series(['saleprice', 'yrsold', 'miscval', 'poolarea','screenporch', 'threessnporch', 'enclosedporch',
                             'wooddecksf', 'openporchsf','garagearea', 'garageyrblt', 'grlivarea', 'lowqualfinsf', 'firstflrsf', 
                             'secondflrsf','totalbsmtsf', 'bsmtunfsf', 'bsmtfinsf1', 'bsmtfinsf2', 'masvnrarea', 
                             'yearbuilt', 'yearremodadd','lotarea', 'lotfrontage'])

#Create series of the categorical variable names

cat_vars = []

for var in prices_df.columns:
    if cont_vars.str.contains(var).any() == False:
        cat_vars.append(var)

cat_vars = pd.Series(cat_vars)

#Check that no variables are in both variable lists

for var in prices_df.columns:
    if cont_vars.str.contains(var).any() & cat_vars.str.contains(var).any():
        print(var)

In [5]:
#Convert categorical variables to numerically coded categories

coded_df = prices_df.copy()
coded_df.sort_values(by='saleprice')
codebook = {}

for var in cat_vars:
    if  coded_df[var].dtype == 'O':
        
        #Create replace dict for each variable
        labels = coded_df[var].unique()
        replace_dict = {k: v for k,v in zip(labels,range(len(labels)))}
        
        #Add var entry to codebook and replace in dataframe
        codebook.update({var: replace_dict})
        coded_df.loc[:,var] = coded_df[var].replace(replace_dict) 

In [6]:
#Boxcox transform our dependent variable, add to list of cont_vars 

coded_df['boxcox_saleprice'] = stats.boxcox(coded_df['saleprice'])[0]
cont_vars = list(cont_vars)
cont_vars.insert(0,'boxcox_saleprice')

In [7]:
#Sort variables by their relevance in the model

#Loop through cat_vars and conduct One-Way Anova across the groups in each cat_var for boxcox_saleprice

import statsmodels.api as sm
from statsmodels.formula.api import ols
F = []
p = []

for var in cat_vars:
    mod_str = 'boxcox_saleprice ~ ' + var
    mod = ols(mod_str, data=coded_df).fit()
    aov_table= sm.stats.anova_lm(mod, typ=2)
    F.append(aov_table['F'][0])
    p.append(aov_table['PR(>F)'][0])
    
#Save results of anova in df with variable names

aov_res = pd.DataFrame(cat_vars, columns=['var'])
aov_res['F'] = F
aov_res['p-value'] = p 
aov_res.sort_values(by='F', ascending=False, inplace=True)

#Create correlation matrix, sort by saleprice
cont_corrs = coded_df[cont_vars].corr()
cont_corrs.sort_values(by='boxcox_saleprice',axis=0, ascending=False, inplace=True)

In [8]:
#Create series of top 10 categorical variables
features = aov_res['var'][:10]

#Append series of continuous variables with greater than 0.5 correlation and remove saleprice vars and secondflrsf
cont_features = list(cont_corrs.loc[cont_corrs['boxcox_saleprice']>0.5, 'boxcox_saleprice'].index)
for var in ['boxcox_saleprice', 'saleprice']: cont_features.remove(var) 
    
features = features.append(pd.Series(cont_features), ignore_index=True)

In [9]:
coded_df.loc[:,features] = coded_df.loc[:,features].fillna(0)

# OLS Regression

In [44]:
#Sort data into dependent and independent variables
Y = coded_df['boxcox_saleprice']
X = coded_df[features]

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=2)

lrm = LinearRegression(fit_intercept=True)

lrm.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.8303756979176486
-----Test set statistics-----
R-squared of the model in test set is: 0.8371229477521631
Mean absolute error of the prediction is: 0.04545957195088056
Mean squared error of the prediction is: 0.004098903185340866
Root mean squared error of the prediction is: 0.0640226771178843
Mean absolute percentage error of the prediction is: 0.5836240732110317


# Lasso Regression

In [42]:
alphas = [np.power(10.0,p) for p in np.arange(-10,40)]

In [46]:
lasso_cv = LassoCV(alphas=alphas, cv=5)

lasso_cv.fit(X_train, y_train)

y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)

print("Best alpha value is: {}".format(lasso_cv.alpha_))
print("R-squared of the model in training set is: {}".format(lasso_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lasso_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 0.0001
R-squared of the model in training set is: 0.8303603333789558
-----Test set statistics-----
R-squared of the model in test set is: 0.8375590691988901
Mean absolute error of the prediction is: 0.04533147983005963
Mean squared error of the prediction is: 0.004087927915574413
Root mean squared error of the prediction is: 0.06393690573975576
Mean absolute percentage error of the prediction is: 0.5820098384416624


# Ridge Regression

In [45]:
ridge_cv = RidgeCV(alphas=alphas, cv=5)

ridge_cv.fit(X_train, y_train)

y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print("Best alpha value is: {}".format(ridge_cv.alpha_))
print("R-squared of the model in training set is: {}".format(ridge_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridge_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 10.0
R-squared of the model in training set is: 0.8303487184245927
-----Test set statistics-----
R-squared of the model in test set is: 0.8376743317480655
Mean absolute error of the prediction is: 0.04526694299771495
Mean squared error of the prediction is: 0.004085027261225347
Root mean squared error of the prediction is: 0.06391421798962534
Mean absolute percentage error of the prediction is: 0.5811784156793333


/home/derek/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# ElasticNet Regression

In [47]:

elasticnet_cv = ElasticNetCV(alphas=alphas, cv=5)

elasticnet_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticnet_cv.predict(X_train)
y_preds_test = elasticnet_cv.predict(X_test)

print("Best alpha value is: {}".format(elasticnet_cv.alpha_))
print("R-squared of the model in training set is: {}".format(elasticnet_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(elasticnet_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 0.0001
R-squared of the model in training set is: 0.8303715402623383
-----Test set statistics-----
R-squared of the model in test set is: 0.8373589634736343
Mean absolute error of the prediction is: 0.04539189354832565
Mean squared error of the prediction is: 0.004092963701667888
Root mean squared error of the prediction is: 0.06397627452163722
Mean absolute percentage error of the prediction is: 0.5827709449067502


# Summary

The models all performed very similarly. The best performance came from the Ridge Regression model where Mean absolute percentage error was 0.001% lower. All other metrics were nearly indistinguishable from one another. 